# Anyco Telecom Customer Support Agent with MCP

In this notebook, we'll build a comprehensive customer support agent for Anyco Telecom, a cellular service provider. This agent will help customer service representatives quickly access customer information, billing details, usage statistics, and resolve common customer inquiries.

## What We're Building

Our customer support agent will be able to:

1. **Customer Account Lookup**: Find customers by phone or email, get complete profiles
2. **Billing & Invoices**: Check current bills, payment history, overdue amounts
3. **Service Plans & Usage**: View current plans, data usage, overage charges
4. **Add-ons & Features**: List active add-ons and available options
5. **Account Troubleshooting**: Check suspension status, recent account changes
6. **Payment Methods**: View and validate payment information
7. **Family Plans**: Manage multi-line accounts and calculate total costs
8. **Agent Tools**: Find high-usage customers, overdue accounts, loyalty opportunities

## MCP Architecture Benefits

Using the Model Context Provider (MCP) pattern allows us to:

- **Separate Database Logic**: All SQL queries and database connections are handled by the MCP server
- **Reusable Tools**: The same tools can be used by multiple agents or applications
- **Scalable**: Easy to add new customer support tools without changing agent code
- **Maintainable**: Database schema changes only require updating the MCP server

Let's get started!

## Strands SDK with MCP Server Architecture
Note how we've now moved all tool definitions into our FastMCP server. We'll see how Strands SDK seamlessly integrates with this server using the Model Context Protocol (MCP).

![Strands SDK Agent & MCP Architecture](../images/workshop_diagrams-strands_and_mcp.drawio.png)

In [ ]:
%pip install mysql-connector-python PyMySQL strands-agents strands-agents-tools sqlalchemy --quiet

In [ ]:
import os
import json
import pandas as pd
from IPython.display import display, HTML, Markdown
from typing import Dict, Any, Optional, List
import mysql.connector
from sqlalchemy import create_engine

## 1. Understanding Our MCP Server

Let's first examine our Telecom Customer Support MCP server that contains all the database tools. These should look familiar from our previous notebook:

In [ ]:
# Display the MCP server implementation
with open('mcp_server_example.py', 'r') as f:
    code = f.read()

# Show just the first part to understand the structure
lines = code.split('\n')
preview_lines = lines[:80]  # Show first 80 lines
preview_code = '\n'.join(preview_lines)

display(Markdown(f"```python\n{preview_code}\n...\n# (Additional tools continue...)\n```"))

Our MCP server includes 25+ specialized tools for customer support, each corresponding to common customer service queries. The tools are organized into logical categories and use parameterized SQL queries to safely interact with the MySQL database.

## 2. Create MCP Client Connection

Now let's create a client connection to our Telecom MCP server:

In [ ]:
from strands.tools.mcp import MCPClient
from mcp import StdioServerParameters, stdio_client

# Create MCP client for our telecom customer support server
telecom_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="python", 
            args=["./mcp_server_example.py"], 
            env=os.environ.copy()
        )
    )
)

print("✅ MCP Client created successfully")

## 3. Explore Available Tools

Let's see what tools are available in our MCP server:

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    print("tools", tools[0])
    print(f"📋 Available Tools: {len(tools)} total\n")
    
    for tool in tools:
        print(f"   • 🔧 {tool.tool_name}: {tool.tool_spec['description'][:30]}...")

## 4. Set Up the Customer Support Agent

Now let's create our intelligent customer support agent with specialized instructions for telecom support:

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from nova_custom_model_provider import NovaCustomModelProvider

# Define the customer support agent's system prompt
agent_instruction = """## Role
You are Alex, a Customer Support Agent for Anyco Telecom, a cellular service provider. You answer questions asked by customers about their current cell phone plan.

## Model Instructions:
- When a customer provides their phone number or email, use the available tools directly with that information
- Most tools accept either 'phone' or 'email' parameters - pass whichever one the customer provided
- You can use find_customer_by_phone or find_customer_by_email to verify customer identity
- Always refer to the customer by their first name, be professional, helpful, and empathetic in your responses
- Provide clear, accurate information about billing, usage, plans, and account status
- For plan questions, show current plans and usage, suggest upgrades if needed
- Always verify customer identity before providing sensitive account information
- Offer solutions and next steps for any issues identified

## Tool Usage:
- All customer data tools accept 'phone' and/or 'email' parameters
- Pass the customer's phone number or email directly to tools like:
  - get_current_bill(phone="555-1001") or get_current_bill(email="customer@email.com")
  - get_current_usage(phone="555-1001")
  - get_payment_history(phone="555-1001")
- You don't need to look up a customer ID first - just use their phone or email

## Billing Best Practices
- For billing questions, show current charges and explain any fees or overages

## Account and Service Best Practices
- For service issues, check account suspension status and recent activity
- Suspended accounts can be due to no valid payment methods or overdue charges - use the correct tools to check for this
- Explain charges and fees in simple terms
- Provide specific dates, amounts, and details when discussing billing
- Offer proactive solutions (e.g., payment plans for overdue accounts)
- Identify upsell opportunities for high-usage customers
- Document any account changes or commitments made

## Response style and format requirements:
- Always respond in valid markdown syntax
- only use a single asterisks (e.g. *) for bullet points
- Provide clear, concise responses focused on solving the customer's issue
- Include relevant account details (amounts, dates, plan names) in your responses
- Format financial information clearly (e.g., "$45.99 due on March 15th")
- Suggest specific next steps or actions when appropriate
"""


### Using a Strands Custom Model Provider
Nova variants are prone to output `<thinking>` tags when progressing through their chain of thought. This is easily addressed in strands by building a [Custom Model Provider](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/model-providers/custom_model_provider/#implementing-a-custom-model-provider). 

Take a look at [`nova_custom_model_provider.py`](./nova_custom_model_provider.py) to see how we extended the Strands BedrockModel class to support tripping these tags out. Swap the below with `BedrockModel` in place of `NovaCustomModelProvider` to see the difference!

In [ ]:
# Initialize the Custom Nova Amazon Bedrock model
model = NovaCustomModelProvider(
    model_id="us.amazon.nova-pro-v1:0",
    max_tokens=4000,
    temperature=0.3,
    top_p=0.9,
    additional_request_fields={
        "inferenceConfig": {
            "topK": 50
        }
    }
)

print("✅ Customer Support Agent configured")

## 5. Test Customer Support Scenarios

Let's test our agent with realistic customer support scenarios:

### Scenario 1: Customer Account Lookup

In [ ]:
# import logging

# # Configure the root strands logger
# logging.getLogger("strands").setLevel(logging.DEBUG)

# # Add a handler to see the logs
# logging.basicConfig(
#     format="%(levelname)s | %(name)s | %(message)s", 
#     handlers=[logging.StreamHandler()]
# )

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()

    # Create the Customer Support Agent
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Customer Support Scenario 1: Account Lookup\n")
    
    response = agent(
        """Hi, I'm calling about my account. My phone number is 555-1001. 
        Can you tell me what plan I'm on and how much data I've used this month?"""
    )
    
    print("🎧 Agent Response:")
    print(response)

### Scenario 2: Billing Inquiry

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Customer Support Scenario 2: Billing Question\n")
    
    response = agent(
        """I received my bill and it's higher than usual. My number is 555-3001. 
        Can you help me understand what I'm being charged for this month?"""
    )
    
    print("🎧 Agent Response:")
    print(response)

### Scenario 3: Service Issue Troubleshooting

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Customer Support Scenario 3: Service Issue\n")
    
    response = agent(
        """My phone service isn't working. I can't make calls or use data. 
        My email is lisa.anderson@icloud.com. Can you check what's wrong with my account?"""
    )
    
    print("🎧 Agent Response:")
    print(response)

### Scenario 4: Family Plan Management

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Customer Support Scenario 4: Family Plan Inquiry\n")
    
    response = agent(
        """I have a family plan and want to see all the lines on my account 
        and what my total monthly cost is. My number is 555-3001."""
    )
    
    print("🎧 Agent Response:")
    print(response)

### Scenario 5: Payment and Discount Information

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Customer Support Scenario 5: Payment & Discounts\n")
    
    response = agent(
        """Can you tell me when my last payment was and what discounts 
        I have on my account? My phone number is 555-5001."""
    )
    
    print("🎧 Agent Response:")
    print(response)

## 6. Agent Tools for Customer Service Representatives

Our MCP server also includes specialized tools for customer service representatives to identify opportunities and manage their workload:

### Find Customers with Overdue Payments

In [ ]:
agent_support_instruction = """## Role
You are an AI Assistant designed to help Customer Support Representatives at Anyco Telecom, a cellular service provider. You provide guidance, suggestions, and procedural support to help representatives assist customers with their cell phone plans and accounts.

## Model Instructions
- If the representative needs data on anything, use the available tools to get them the data they need.
- Remember than each customer has a CustomerID unique to them. You can find them by using lookup_account_by_number or lookup_account_by_email tools
- Remind representatives to address customers by their first name and maintain professional, helpful, and empathetic communication
- Help reps locate and interpret information about billing, usage, plans, and account status
- For billing inquiries, guide reps to show current charges and help them explain any fees or overages to customers
- For service issues, prompt reps to check account suspension status and recent activity
- For plan questions, help reps review current plans and usage, suggest when upgrades might be appropriate
- Always emphasize the importance of verifying customer identity before sharing sensitive account information
- Suggest solutions and next steps that representatives can offer to resolve customer issues

## Representative Guidance Best Practices
- Remind reps that suspended accounts are often due to no valid payment methods or overdue charges - guide them to use the appropriate tools to investigate
- Help representatives translate charges and fees into simple, customer-friendly language
- Prompt reps to reference specific dates, amounts, and details when discussing billing with customers
- Suggest proactive solutions reps can offer (e.g., payment plans for overdue accounts, plan adjustments)
- Alert representatives to potential upsell opportunities for high-usage customers
- Remind reps to document any account changes or commitments made during the call

## Response style and format requirements:
- Always format guidance using valid markdown syntax
- Provide clear, actionable advice that helps representatives solve customer issues effectively
- Suggest specific account details reps should reference (amounts, dates, plan names) in their responses
- Recommend how reps should format financial information for customers (e.g., "Tell the customer: '$45.99 due on March 15th'")
- Propose concrete next steps or actions that representatives can offer to customers
"""

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_support_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("🔍 Agent Tool: Find Overdue Customers\n")
    
    response = agent(
        """I need to see a list of customers with overdue payments for collections follow-up. 
        Can you show me the customers who have overdue invoices?"""
    )
    
    print("📊 Agent Response:")
    print(response)

### Find High Usage Customers (Upsell Opportunities)

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("🔍 Agent Tool: Find High Usage Customers\n")
    
    response = agent(
        """Show me customers who are using over 80% of their data allowance. 
        These might be good candidates for plan upgrades."""
    )
    
    print("📊 Agent Response:")
    print(response)

### Find Loyalty Program Eligible Customers

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("🔍 Agent Tool: Find Loyalty Eligible Customers\n")
    
    response = agent(
        """I want to identify any long-term customers who might be eligible for loyalty discounts. 
        Can you find customers who have been with us for over a year but don't have loyalty discounts?"""
    )
    
    print("📊 Agent Response:")
    print(response)

## 7. Advanced Customer Support Scenarios

Let's test some more complex scenarios that require multiple tool calls:

### Complex Billing Investigation

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Advanced Scenario: Complex Billing Investigation\n")
    
    response = agent(
        """A customer with phone number 555-5001 is disputing their bill. They say they've been 
        charged for services they didn't use. Can you do a complete investigation? I need to see:
        1. Their current bill breakdown
        2. Their usage this month
        3. Any overage charges
        4. Their active add-ons
        5. Recent account changes
        
        Please provide a comprehensive analysis."""
    )
    
    print("🎧 Agent Response:")
    print(response)

### Business Account Review

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📞 Advanced Scenario: Business Account Review\n")
    
    response = agent(
        """I have a business customer (phone: 555-0110) who wants a complete account review 
        for their monthly business meeting. Can you provide:
        1. Business account summary
        2. All lines and their usage
        3. Total monthly costs
        4. Any available discounts they might qualify for
        5. Payment history for the last 3 months
        
        Format this as a professional account summary."""
    )
    
    print("🎧 Agent Response:")
    print(response)

## 8. Performance and Analytics

Let's demonstrate how the agent can help with performance analytics and customer insights:

In [ ]:
with telecom_mcp_client:
    tools = telecom_mcp_client.list_tools_sync()
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    print("📊 Performance Analytics Request\n")
    
    response = agent(
        """I need to prepare a report for my manager. Can you help me identify:
        1. All customers with overdue payments (for collections priority)
        2. High data usage customers over 80% (for upsell campaigns)
        3. Long-term customers eligible for loyalty programs
        
        Please provide counts and key insights for each category."""
    )
    
    print("📈 Agent Response:")
    print(response)

# Conclusion

In this notebook, we've built a comprehensive customer support agent for Anyco Telecom using the Model Context Provider (MCP) architecture. Our implementation demonstrates several key advantages:

## What We Accomplished

1. **Complete Customer Support Coverage**: Our agent can handle the full spectrum of customer inquiries from basic account lookups to complex billing investigations

2. **Database Integration**: All customer data is accessed through secure, parameterized SQL queries that prevent injection attacks

3. **Intelligent Tool Selection**: The agent automatically chooses the right combination of tools based on customer needs

4. **Professional Communication**: Responses are formatted appropriately for customer service representatives

5. **Analytics and Insights**: Built-in tools help identify business opportunities and manage workload priorities

## MCP Architecture Benefits for Customer Support

- **Modularity**: Database queries are separated from agent logic, making maintenance easier
- **Reusability**: The same MCP server can power multiple customer support applications
- **Scalability**: New customer support tools can be added without changing agent code
- **Security**: Database credentials and connection logic are isolated in the MCP server
- **Consistency**: All agents use the same data access patterns and business logic

## Real-World Applications

This architecture is ideal for:
- **Call Center Operations**: Agents can quickly access comprehensive customer information
- **Self-Service Portals**: Customers can get instant answers to common questions
- **Mobile Support Apps**: Field technicians can access customer data on mobile devices
- **Business Intelligence**: Management can analyze customer patterns and support metrics
- **Training Systems**: New representatives can practice with realistic customer scenarios

The MCP pattern provides a robust foundation for building sophisticated customer support systems that can evolve with business needs while maintaining consistency and reliability.